# Product Description Generation
In this project, you can generate descriptions of various apparel items like shirts, jeans, kurtas using RNNs. The descriptions of 10400 products for each apparel item has been used fro jabong.com to train the neural network. The Neural Network built will generate a product description for a product from the given sets of apparels i.e. shirts, jeans, shoes, watches and kurtas.
## Get the Data
The data consists of 10400 descriptions of different jeans obtained from jabong.com with distinguishing features.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/t-shirts.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[:]

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 10308
Number of scenes: 20484
Average number of sentences in each scene: 0.9999511814098809
Number of lines: 40967
Average number of words in each line: 31.192374350086656

The sentences 0 to 10:
﻿HERE&NOW Charcoal Grey Regular Fit Faded Casual Shirt: Charcoal grey faded casual shirt, has a mandarin collar, button placket, long sleeves, curved hem


Jack & Jones Rust Solid Slim Fit Casual Shirt: Breathe a new life into your casual wardrobe by adding this rust coloured casual shirt for men by Jack & Jones. This slim-fit shirt will ensure a comfortable fit, courtesy its cotton fabric. Furthermore, this shirt will go well with jeans and loafers.


Wrangler Multicoloured Checked Regular Fit Casual Shirt: Multi in colour, this shirt from Wrangler is a basic essential in every urbane man’s casual collection. High on style, this lightweight cotton shirt will be an absolute delight to wear. This shirt can be teamed with a pair of black trousers

## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [3]:
import numpy as np
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    text = list(set(text))   
    index = range(len(text))    
    int_to_vocab = dict(zip(index, text))   
    vocab_to_int = dict(zip(text, index))       
    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    token = {}
    token['.']="||period||"
    token[',']="||comma||"
    token['"']="||quotation_mark||"
    token[';']="||semicolon||"
    token['!']="||exclamation_mark||"
    token['?']="||question_mark||"
    token['(']="||left_parantheses||"
    token[')']="||right_parantheses||"
    token['--'] = "||dash||"
    token['\n'] = "||return||"
    
    return token

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0
Default GPU Device: /gpu:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [8]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    INPUT = tf.placeholder(tf.int32, shape=(None,None), name="input")
    TARGETS = tf.placeholder(tf.int32, shape=(None,None), name='targets')
    LEARNING_RATE = tf.placeholder(tf.float32, shape=None, name='learning_rate')
    return INPUT, TARGETS, LEARNING_RATE


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [9]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    lstm1 = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    lstm2 = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    lstm3 = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    lstm4 = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    cell = tf.contrib.rnn.MultiRNNCell([lstm1,lstm2,lstm3,lstm4])
    initial_state = cell.zero_state(batch_size, tf.float32)
    initial_state = tf.identity(initial_state, name='initial_state')
   
     
    return cell, initial_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [10]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensionstea
    :return: Embedded input.
    """
    # TODO: Implement Function
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    return embed


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [11]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    Outputs, States = tf.nn.dynamic_rnn(cell, inputs,dtype='float32')
    FinalState = tf.identity(States,name="final_state")
    return Outputs, FinalState


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [12]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    embed = get_embed(input_data, vocab_size, embed_dim)
    outputs, final_state = build_rnn(cell, embed)
    logits = tf.contrib.layers.fully_connected(outputs, vocab_size, activation_fn=None,
                                               weights_initializer=tf.truncated_normal_initializer(stddev=0.1),
                                               biases_initializer=tf.zeros_initializer())
    return logits, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For example, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [13]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    num_batches = int(len(int_text) / (batch_size * seq_length))
    in_data = np.array(int_text[: num_batches * batch_size * seq_length])
    out_data = np.array(int_text[1: num_batches * batch_size * seq_length + 1])
    out_data[-1] = in_data[0]

    in_batches = np.split(in_data.reshape(batch_size, -1), num_batches, 1)
    out_batches = np.split(out_data.reshape(batch_size, -1), num_batches, 1)
    return np.array(list(zip(in_batches, out_batches)))


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [14]:
# Number of Epochs
num_epochs = 3000
# Batch Size
batch_size = 512
# RNN Size
rnn_size = 256
# Embedding Dimension Size
embed_dim = 256
# Sequence Length
seq_length = 16
# Learning Rate
learning_rate = 0.01
# Show stats for every n number of batches
show_every_n_batches = 32

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './savetshirts'

### Build the Graph
Build the graph using the neural network you implemented.

In [15]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [16]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')
    

Epoch   0 Batch    0/175   train_loss = 8.793
Epoch   0 Batch   32/175   train_loss = 5.839
Epoch   0 Batch   64/175   train_loss = 5.828
Epoch   0 Batch   96/175   train_loss = 5.847
Epoch   0 Batch  128/175   train_loss = 5.805
Epoch   0 Batch  160/175   train_loss = 5.795
Epoch   1 Batch   17/175   train_loss = 5.751
Epoch   1 Batch   49/175   train_loss = 5.732
Epoch   1 Batch   81/175   train_loss = 5.336
Epoch   1 Batch  113/175   train_loss = 4.812
Epoch   1 Batch  145/175   train_loss = 4.154
Epoch   2 Batch    2/175   train_loss = 3.746
Epoch   2 Batch   34/175   train_loss = 3.471
Epoch   2 Batch   66/175   train_loss = 3.118
Epoch   2 Batch   98/175   train_loss = 2.825
Epoch   2 Batch  130/175   train_loss = 2.644
Epoch   2 Batch  162/175   train_loss = 2.407
Epoch   3 Batch   19/175   train_loss = 2.308
Epoch   3 Batch   51/175   train_loss = 2.125
Epoch   3 Batch   83/175   train_loss = 2.160
Epoch   3 Batch  115/175   train_loss = 2.226
Epoch   3 Batch  147/175   train_l

Epoch  32 Batch  128/175   train_loss = 1.192
Epoch  32 Batch  160/175   train_loss = 1.219
Epoch  33 Batch   17/175   train_loss = 1.197
Epoch  33 Batch   49/175   train_loss = 1.222
Epoch  33 Batch   81/175   train_loss = 1.190
Epoch  33 Batch  113/175   train_loss = 1.222
Epoch  33 Batch  145/175   train_loss = 1.198
Epoch  34 Batch    2/175   train_loss = 1.218
Epoch  34 Batch   34/175   train_loss = 1.214
Epoch  34 Batch   66/175   train_loss = 1.238
Epoch  34 Batch   98/175   train_loss = 1.234
Epoch  34 Batch  130/175   train_loss = 1.227
Epoch  34 Batch  162/175   train_loss = 1.209
Epoch  35 Batch   19/175   train_loss = 1.225
Epoch  35 Batch   51/175   train_loss = 1.165
Epoch  35 Batch   83/175   train_loss = 1.266
Epoch  35 Batch  115/175   train_loss = 1.308
Epoch  35 Batch  147/175   train_loss = 1.197
Epoch  36 Batch    4/175   train_loss = 1.224
Epoch  36 Batch   36/175   train_loss = 1.171
Epoch  36 Batch   68/175   train_loss = 1.182
Epoch  36 Batch  100/175   train_l

Epoch  65 Batch   81/175   train_loss = 1.152
Epoch  65 Batch  113/175   train_loss = 1.190
Epoch  65 Batch  145/175   train_loss = 1.148
Epoch  66 Batch    2/175   train_loss = 1.173
Epoch  66 Batch   34/175   train_loss = 1.199
Epoch  66 Batch   66/175   train_loss = 1.195
Epoch  66 Batch   98/175   train_loss = 1.211
Epoch  66 Batch  130/175   train_loss = 1.212
Epoch  66 Batch  162/175   train_loss = 1.189
Epoch  67 Batch   19/175   train_loss = 1.171
Epoch  67 Batch   51/175   train_loss = 1.132
Epoch  67 Batch   83/175   train_loss = 1.230
Epoch  67 Batch  115/175   train_loss = 1.291
Epoch  67 Batch  147/175   train_loss = 1.155
Epoch  68 Batch    4/175   train_loss = 1.191
Epoch  68 Batch   36/175   train_loss = 1.141
Epoch  68 Batch   68/175   train_loss = 1.161
Epoch  68 Batch  100/175   train_loss = 1.172
Epoch  68 Batch  132/175   train_loss = 1.122
Epoch  68 Batch  164/175   train_loss = 1.149
Epoch  69 Batch   21/175   train_loss = 1.141
Epoch  69 Batch   53/175   train_l

Epoch  98 Batch   34/175   train_loss = 1.175
Epoch  98 Batch   66/175   train_loss = 1.193
Epoch  98 Batch   98/175   train_loss = 1.182
Epoch  98 Batch  130/175   train_loss = 1.177
Epoch  98 Batch  162/175   train_loss = 1.166
Epoch  99 Batch   19/175   train_loss = 1.165
Epoch  99 Batch   51/175   train_loss = 1.120
Epoch  99 Batch   83/175   train_loss = 1.211
Epoch  99 Batch  115/175   train_loss = 1.259
Epoch  99 Batch  147/175   train_loss = 1.144
Epoch 100 Batch    4/175   train_loss = 1.186
Epoch 100 Batch   36/175   train_loss = 1.133
Epoch 100 Batch   68/175   train_loss = 1.164
Epoch 100 Batch  100/175   train_loss = 1.156
Epoch 100 Batch  132/175   train_loss = 1.113
Epoch 100 Batch  164/175   train_loss = 1.148
Epoch 101 Batch   21/175   train_loss = 1.126
Epoch 101 Batch   53/175   train_loss = 1.144
Epoch 101 Batch   85/175   train_loss = 1.175
Epoch 101 Batch  117/175   train_loss = 1.188
Epoch 101 Batch  149/175   train_loss = 1.174
Epoch 102 Batch    6/175   train_l

Epoch 130 Batch  162/175   train_loss = 1.171
Epoch 131 Batch   19/175   train_loss = 1.165
Epoch 131 Batch   51/175   train_loss = 1.103
Epoch 131 Batch   83/175   train_loss = 1.221
Epoch 131 Batch  115/175   train_loss = 1.259
Epoch 131 Batch  147/175   train_loss = 1.152
Epoch 132 Batch    4/175   train_loss = 1.160
Epoch 132 Batch   36/175   train_loss = 1.129
Epoch 132 Batch   68/175   train_loss = 1.154
Epoch 132 Batch  100/175   train_loss = 1.159
Epoch 132 Batch  132/175   train_loss = 1.120
Epoch 132 Batch  164/175   train_loss = 1.124
Epoch 133 Batch   21/175   train_loss = 1.126
Epoch 133 Batch   53/175   train_loss = 1.142
Epoch 133 Batch   85/175   train_loss = 1.172
Epoch 133 Batch  117/175   train_loss = 1.163
Epoch 133 Batch  149/175   train_loss = 1.187
Epoch 134 Batch    6/175   train_loss = 1.182
Epoch 134 Batch   38/175   train_loss = 1.118
Epoch 134 Batch   70/175   train_loss = 1.161
Epoch 134 Batch  102/175   train_loss = 1.172
Epoch 134 Batch  134/175   train_l

...

Epoch 2870 Batch   54/175   train_loss = 1.336
Epoch 2870 Batch   86/175   train_loss = 1.337
Epoch 2870 Batch  118/175   train_loss = 1.330
Epoch 2870 Batch  150/175   train_loss = 1.249
Epoch 2871 Batch    7/175   train_loss = 1.289
Epoch 2871 Batch   39/175   train_loss = 1.200
Epoch 2871 Batch   71/175   train_loss = 1.305
Epoch 2871 Batch  103/175   train_loss = 1.286
Epoch 2871 Batch  135/175   train_loss = 1.299
Epoch 2871 Batch  167/175   train_loss = 1.355
Epoch 2872 Batch   24/175   train_loss = 1.220
Epoch 2872 Batch   56/175   train_loss = 1.310
Epoch 2872 Batch   88/175   train_loss = 1.332
Epoch 2872 Batch  120/175   train_loss = 1.362
Epoch 2872 Batch  152/175   train_loss = 1.309
Epoch 2873 Batch    9/175   train_loss = 1.322
Epoch 2873 Batch   41/175   train_loss = 1.323
Epoch 2873 Batch   73/175   train_loss = 1.295
Epoch 2873 Batch  105/175   train_loss = 1.333
Epoch 2873 Batch  137/175   train_loss = 1.249
Epoch 2873 Batch  169/175   train_loss = 1.279
Epoch 2874 Ba

Epoch 2902 Batch   54/175   train_loss = 1.331
Epoch 2902 Batch   86/175   train_loss = 1.325
Epoch 2902 Batch  118/175   train_loss = 1.299
Epoch 2902 Batch  150/175   train_loss = 1.275
Epoch 2903 Batch    7/175   train_loss = 1.277
Epoch 2903 Batch   39/175   train_loss = 1.196
Epoch 2903 Batch   71/175   train_loss = 1.286
Epoch 2903 Batch  103/175   train_loss = 1.259
Epoch 2903 Batch  135/175   train_loss = 1.230
Epoch 2903 Batch  167/175   train_loss = 1.305
Epoch 2904 Batch   24/175   train_loss = 1.200
Epoch 2904 Batch   56/175   train_loss = 1.255
Epoch 2904 Batch   88/175   train_loss = 1.303
Epoch 2904 Batch  120/175   train_loss = 1.236
Epoch 2904 Batch  152/175   train_loss = 1.222
Epoch 2905 Batch    9/175   train_loss = 1.260
Epoch 2905 Batch   41/175   train_loss = 1.295
Epoch 2905 Batch   73/175   train_loss = 1.259
Epoch 2905 Batch  105/175   train_loss = 1.284
Epoch 2905 Batch  137/175   train_loss = 1.200
Epoch 2905 Batch  169/175   train_loss = 1.264
Epoch 2906 Ba

Epoch 2934 Batch   54/175   train_loss = 1.306
Epoch 2934 Batch   86/175   train_loss = 1.316
Epoch 2934 Batch  118/175   train_loss = 1.305
Epoch 2934 Batch  150/175   train_loss = 1.248
Epoch 2935 Batch    7/175   train_loss = 1.261
Epoch 2935 Batch   39/175   train_loss = 1.216
Epoch 2935 Batch   71/175   train_loss = 1.296
Epoch 2935 Batch  103/175   train_loss = 1.254
Epoch 2935 Batch  135/175   train_loss = 1.225
Epoch 2935 Batch  167/175   train_loss = 1.309
Epoch 2936 Batch   24/175   train_loss = 1.191
Epoch 2936 Batch   56/175   train_loss = 1.303
Epoch 2936 Batch   88/175   train_loss = 1.302
Epoch 2936 Batch  120/175   train_loss = 1.240
Epoch 2936 Batch  152/175   train_loss = 1.235
Epoch 2937 Batch    9/175   train_loss = 1.267
Epoch 2937 Batch   41/175   train_loss = 1.299
Epoch 2937 Batch   73/175   train_loss = 1.303
Epoch 2937 Batch  105/175   train_loss = 1.331
Epoch 2937 Batch  137/175   train_loss = 1.239
Epoch 2937 Batch  169/175   train_loss = 1.311
Epoch 2938 Ba

Epoch 2966 Batch   54/175   train_loss = 1.309
Epoch 2966 Batch   86/175   train_loss = 1.335
Epoch 2966 Batch  118/175   train_loss = 1.327
Epoch 2966 Batch  150/175   train_loss = 1.269
Epoch 2967 Batch    7/175   train_loss = 1.292
Epoch 2967 Batch   39/175   train_loss = 1.223
Epoch 2967 Batch   71/175   train_loss = 1.286
Epoch 2967 Batch  103/175   train_loss = 1.278
Epoch 2967 Batch  135/175   train_loss = 1.227
Epoch 2967 Batch  167/175   train_loss = 1.315
Epoch 2968 Batch   24/175   train_loss = 1.218
Epoch 2968 Batch   56/175   train_loss = 1.290
Epoch 2968 Batch   88/175   train_loss = 1.337
Epoch 2968 Batch  120/175   train_loss = 1.264
Epoch 2968 Batch  152/175   train_loss = 1.252
Epoch 2969 Batch    9/175   train_loss = 1.287
Epoch 2969 Batch   41/175   train_loss = 1.301
Epoch 2969 Batch   73/175   train_loss = 1.279
Epoch 2969 Batch  105/175   train_loss = 1.350
Epoch 2969 Batch  137/175   train_loss = 1.244
Epoch 2969 Batch  169/175   train_loss = 1.319
Epoch 2970 Ba

Epoch 2998 Batch   54/175   train_loss = 1.295
Epoch 2998 Batch   86/175   train_loss = 1.337
Epoch 2998 Batch  118/175   train_loss = 1.345
Epoch 2998 Batch  150/175   train_loss = 1.273
Epoch 2999 Batch    7/175   train_loss = 1.290
Epoch 2999 Batch   39/175   train_loss = 1.219
Epoch 2999 Batch   71/175   train_loss = 1.288
Epoch 2999 Batch  103/175   train_loss = 1.266
Epoch 2999 Batch  135/175   train_loss = 1.234
Epoch 2999 Batch  167/175   train_loss = 1.334
Model Trained and Saved


## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [19]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    input_tensor = loaded_graph.get_tensor_by_name('input:0')
    initial_state_tensor = loaded_graph.get_tensor_by_name('initial_state:0')
    final_state_tensor = loaded_graph.get_tensor_by_name('final_state:0')
    probs_tensor = loaded_graph.get_tensor_by_name('probs:0')
    
    return input_tensor, initial_state_tensor, final_state_tensor, probs_tensor



tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [20]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    next_word = np.random.choice(len(int_to_vocab), p=probabilities)
    prediction = int_to_vocab[next_word]
    return prediction




tests.test_pick_word(pick_word)

Tests Passed


## Generate Product Description
This will generate the Description for your product.  Set `gen_length` to the length of product description you want to generate.

In [21]:
def generate_description(feature):

    gen_length = 100
    # Sentences generation setup
    gen_sentences = [feature]
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})
    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})

        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)

    # Remove tokens
    product_desc = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        product_desc = product_desc.replace(' ' + token.lower(), key)
    product_desc = product_desc.replace('\n ', '\n')
    product_desc = product_desc.replace('( ', '(')



    product_desc = product_desc.split("\n\n")[0]
    prod_title = product_desc.split(":")[0]
    try:
        desc = product_desc.split(":")[1]
    except:
        product_desc = product_desc + ": "
        desc = product_desc.split(":")[1]
    if len(prod_title.split(" ")) <= 10:
        if len(desc.split(" ")) >= 2:
            return product_desc
    else:
        x = generate_description(feature)
        return x


In [22]:
def get_eligibility(desc,feature,feature_list):
    counter = 0
    n = 0
    list_len = len(feature_list)
    for ftr in feature_list:
        if ftr == feature:
            ftr_index = n
            break
        else:
            n+=1
    
    every_other_ftr = []
    
    for i in range(1,(list_len)):
        every_other_ftr.append(feature_list[(i+ftr_index)%list_len])
    for desn in desc:
        flag = 0
        for features in every_other_ftr:
            for keys in desn.split(":")[0].split(" "):
                if features == keys:
                    for keys1 in desn.split(":")[1].split(" "):
                        if features == keys1:
                            flag+=1
                            break
        if flag == (list_len-1):
            counter = counter + 1
    
    elig = counter/10.0
    return elig

In [23]:
def get_prime_word(featurestr):
    
    feature_list = featurestr.lower().split(" ")
    n = 0
    l = len(feature_list)
    eligibility = []
    for each_feature in feature_list:
        n = 1
        descl = []
        for i in range(0,10):
            desc = generate_description(each_feature)
            descl.insert(i,desc)
        eligibility.insert(n,get_eligibility(descl,each_feature,feature_list))
        n = n+1
    lpp = len(eligibility)
    fstelig = eligibility[0]
    fl = 1
    for nm in range(1,lpp):
        if eligibility[nm] == fstelig:
            fl+=1
    if fl == lpp:
        prime_words = feature_list
    else:
        prime_words = [feature_list for _,feature_list in sorted(zip(eligibility,feature_list),reverse=True)] 
    
    m = eligibility[0]
    fl = 1
    for nm in range(1,lpp):
        if eligibility[nm] == m:
            fl+=1
    if fl == lpp:
        flfl = 0
    else:
        eligibility.sort(reverse=True)
    return prime_words



In [24]:
#main function    
import time
import numpy as np
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)
    
    # Sentences generation setup
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})
    gen_length = 100

    features_str = "puma black t-shirt"
    start_time = time.time()
    ftrs = features_str.split(" ")
    l = len(ftrs)
    prime_words= get_prime_word(features_str)
    prime_word = prime_words[0]
    lp = len(prime_words)
    
         
        ### TO Implement############################################################################
        
        #num_prime_desc = 200  
        #num_other_keyword = 100
        #temp = [num_prime_desc + num_keyword * num_other_keyword, 2]
        
        #--------------------
        #|descrp|num_keywords|
        #---------------------
        #|       |           |
        #---------------------
        
        
        ##TO Do
        # 1. select the description which has highest number of keywords
        # 2. In case of multiple descrips have same number of keywords, select the first
        # 3. In case none of the description has all the keywords, select the best possible and 
        #    Augment it with the sentence related to missing keywords from the descriptions directly related 
        #    to that particular keyword
        
        
        # Actual Code Will Go Here
        
    
    first_desc = generate_description(prime_word)
    first_title = first_desc.split(":")[0]
    first_desc_only = first_desc.split(":")[1]
    flag = 0
    for index in range(0,lp):
        
        if prime_words[index] in first_title and prime_words[index] in first_desc_only:
            flag = flag + 1
            
            
            
    desc_array = np.array([first_desc,flag])
    for i in range(1,10):

        desc = generate_description(prime_words[0])
    

        title = desc.split(":")[0]
        desc_only = desc.split(":")[1]

        flag = 0
        for indx in range(0,lp):
            
            if prime_words[indx] in title and prime_words[indx] in desc_only:
                flag+=1


        new_array = np.array([desc,flag])
        desc_array = np.vstack([desc_array,new_array])    

    for ix in range(1,lp):
        
        for i in range(0,5):

            desc = generate_description(prime_words[ix])

            title = desc.split(":")[0]
            desc_only = desc.split(":")[1]

            flag = 0
            for indx in range(0,lp):
                
                if prime_words[indx] in title and prime_words[indx] in desc_only:
                    flag+=1
                
            new_array = np.array([desc,flag])
            desc_array = np.vstack([desc_array,new_array])    
    
            
    max = 0   
    desc_op = ""
    for all in desc_array:
        if(int(all[1])>=max):
            desc_op = all[0]
            max = int(all[1])
    import webcolors
    import helper

    def get_brandname():

        data_dir = './data/simpsons/t-shirts.txt'
        text = helper.load_data(data_dir)
        # Ignore notice, since we don't use it for analysing the data
        text = text[:]
        brands = []
        alldesc = text.split("\n\n")
        for eachdesc in alldesc:
            eachtitle = eachdesc.split(":")[0]
            brandname = eachtitle.split(" ")[0]
            brandname = brandname.replace("\n",'')
            brandname = brandname.replace("\ufeff",'')
            brandname = brandname.lower()
            brands.append(brandname)
        brands = set(brands)
        brands = list(brands)

        brands.remove('the')
        brands.remove('')
        brands.remove('none')
        brands.remove('blend')
        brands.remove('design')
        brands.remove('true')
        brands.remove('all')
        clrlst = get_colors()
        brands = list(set(brands)-set(clrlst))


        return brands

    def get_colors():
        list_of_colors = []
        listcolors = webcolors.CSS3_NAMES_TO_HEX
        for all in listcolors:
            cl = all.split(":")[0]
            list_of_colors.append(cl)
        list_of_colors.append('mauve')
        list_of_colors.append('wine')
        return list_of_colors

    ###############################################################################################################################################
    ############################################################################################################################################3
    ##################################################################################################################################


    #Getting brand names from text files to identify brand in the features string
    brandnames = get_brandname()
    brand_instr = ''
    for allbrands in brandnames:
        for allwords in features_str.split(" "):
            if allbrands == allwords:
                brand_instr = allbrands
                break



    #getting all colors to identify the color in the features string
    colorlist = get_colors()
    color_instr = ''
    for allcolors in colorlist:
        for allwords in features_str.split(" "):
            if allcolors == allwords:
                color_instr = allcolors
                break
            else:
                for allword in desc_op.split(":")[0].split(" "):
                    if allcolors == allword:
                        color_instr = allcolors
                        break
    

    # to append the brand name in the title if not present

    allwordsintitle = desc_op.split(":")[0]

    for allwords in allwordsintitle.split(" "):
        if allwords == color_instr:
            flg = 0
        else:
            allwordsintitle = color_instr + " " + allwordsintitle
            title = desc_op.split(":")[0]
            allwordsintitle = title.replace(desc_op.split(":")[0],allwordsintitle)
            break

    for allwords in allwordsintitle.split(" "):
        if allwords == brand_instr:
            flg = 0
        else:
            allwordsintitle = brand_instr + " " + allwordsintitle
            title = desc_op.split(":")[0]
            allwordsintitle = title.replace(desc_op.split(":")[0],allwordsintitle)
            break


    #to replace brand names and colours in the description 

    allwordsindescription = desc_op.split(":")[1]

    for brandnm in brandnames:
        for allwords in allwordsintitle.split(" "):
            if brandnm == allwords or brandnm+"." == allwords or brandnm+"," == allwords or brandnm+";" == allwords:
                allwordsintitle = allwordsintitle.replace(brandnm,brand_instr)
        for allwords in allwordsindescription.split(" "):
            if brandnm == allwords or brandnm+"." == allwords or brandnm+"," == allwords or brandnm+";" == allwords:
                allwordsindescription = allwordsindescription.replace(brandnm,brand_instr)


    desc_op = desc_op.replace(desc_op.split(":")[0],allwordsintitle)
    desc_op = desc_op.replace(desc_op.split(":")[1],allwordsindescription)
    

    for clrs in colorlist:
        for allwords in allwordsintitle.split(" "):
            if clrs == allwords or clrs+"." == allwords or clrs+"," == allwords or clrs+";" == allwords:
                allwordsintitle = allwordsintitle.replace(clrs,color_instr)
        for allwords in allwordsindescription.split(" "):
            if clrs == allwords or clrs+"." == allwords or clrs+"," == allwords or clrs+";" == allwords:
                allwordsindescription = allwordsindescription.replace(clrs,color_instr)

    def remove_duplicates(li):
        my_set = set()
        res = []
        for e in li:
            if e not in my_set:
                res.append(e)
                my_set.add(e)
        #
        return res

    allwordsintitlelist = allwordsintitle.split(" ")
    allwordsintitlelist = remove_duplicates(allwordsintitlelist)
    allwordsintitle = ' '.join(allwordsintitlelist)

    desc_op = desc_op.replace(desc_op.split(":")[0],allwordsintitle)
    desc_op = desc_op.replace(desc_op.split(":")[1],allwordsindescription)
    final_time = time.time()
    print("time required to generate the description is "+str(final_time - start_time))
    print("\n\nFinal description :")
    print(desc_op)




INFO:tensorflow:Restoring parameters from ./savetshirts
time required to generate the description is 463.8688716888428


Final description :
puma black rbr round neck t-shirt: black in colour, this round-neck t-shirt from puma is a steal and minimal design. not this t-shirt a wardrobe must-have. showcasing a fabulous interplay into your casual style wearing this designer t-shirt for men by puma jeans. this regular-fit t-shirt will ensure a comfortable fit. team this t-shirt with black jeans and black high-ankle. �bullet bike� graphic print on the front pockets area regular fit cotton fabric for improved air circulation. created for an urbane character all over and an eye-catching textured design that make it highly appealing
